In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb # 브라우져를 제어하기 위한 도구
from selenium.webdriver.common.keys import Keys # 키보드를 제어하는 도구
import time # 시간 딜레이를 위한 모듈
import pandas as pd
op = wb.ChromeOptions()
op.add_argument('headless')
op.add_argument('disable-gpu')
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from flask import Flask
from flask import request, redirect

In [2]:
e_data = pd.read_csv('data/감정단어 - 시트1.csv')
e_data_c = e_data.copy()
word_data = e_data_c['관련된 단어']
word_data1 = word_data[0].replace(' ','').split(",")
word_data2  = word_data[1].strip().split(",")
word_data3 = word_data[2].replace(' ','').split(",") 
word_data4 = word_data[3].replace(' ','').split(",")  
word_data5 = word_data[4].replace(' ','').split(",")  
word_data6 = word_data[5].replace(' ','').split(",")  
word_data7 = word_data[6].strip().replace(' ','').split(",")   

In [3]:
e_data2 = pd.read_csv('la3.csv')
e_data2_c = e_data2.copy()
X = e_data2_c.loc[:,'가사']
y = e_data2_c.loc[:,'라벨']
x_train,x_test,y_train,y_test = train_test_split(X,y,
                                               random_state = 50)

In [4]:
X = e_data2_c.loc[:,'가사']
y = e_data2_c.loc[:,'라벨']
x_train,x_test,y_train,y_test = train_test_split(X,y,
                                               random_state = 50)

In [5]:
global word_cv1
word_cv1 = CountVectorizer(max_df=100, min_df=3)
word_cv1.fit(x_train)
x_train_transformed1 = word_cv1.transform(x_train)
x_test_transformed1 = word_cv1.transform(x_test)
global word_logi1
word_logi1 = LogisticRegression(C=0.001,max_iter=1000000,random_state=50)
word_logi1.fit(x_train_transformed1,y_train)

LogisticRegression(C=0.001, max_iter=1000000, random_state=50)

In [6]:
word_logi1.score(x_test_transformed1,y_test)

0.7516778523489933

In [7]:
global e1,e2,e3,e4,e5,e6,e7

In [ ]:
app = Flask(__name__) 

@app.route('/analysis', methods=['GET','POST'])
def index():  
    global e1,e2,e3,e4,e5,e6,e7
    e1,e2,e3,e4,e5,e6,e7 = 0,0,0,0,0,0,0
    #title = request.args['title']
    #singer = request.args['singer']   
    
    if request.method == 'GET':
        title = request.args['title']
        singer = request.args['singer']        
    else:
        title = request.form['title']
        singer = request.form['singer']
    search=f'{singer}&{title}'
    url1 = f'https://www.genie.co.kr/search/searchMain?query={search}'      
    driver = wb.Chrome('chromedriver.exe',chrome_options=op)  
    driver.get(url1)
    time.sleep(1)
    driver.find_element_by_css_selector('.list>.link>.btn-basic').click()
    time.sleep(1)
    html = bs(driver.page_source, 'html.parser')
    a = html.select('#pLyrics p')[0]
    b = word_cv1.transform(a)
    pre = str (word_logi1.predict(b)[0])
    time.sleep(1)
    driver.quit()
    str1 = a.text.replace('\t','').replace('\n','')
    for k in word_data1:
        if k in str1:
            e1+=1
    for k in word_data2:
        if k in str1:
            e2+=1
    for k in word_data3:
            if k in str1:
                e3+=1
    for k in word_data4:
            if k in str1:
                e4+=1
    for k in word_data5:
            if k in str1:
                e5+=1
    for k in word_data6:
            if k in str1:
                e6+=1
    for k in word_data7:
            if k in str1:
                e7+=1
    header ={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36'         
    }
    driver1 = wb.Chrome('chromedriver.exe',chrome_options=op)
        #driver1 = wb.Chrome()
    url2 = 'https://trycolors.com/'
    driver1.get(url2)
    time.sleep(1)
        #분노(빨강):1,놀람(노랑):2,행복(초록):3,슬픔(파랑):4,편안(하늘):5,순수(하양):6,우울(검정):7
    for k in range(e2):#노랑
            driver1.find_element_by_css_selector('.palette_stack>.input_color>.mdl-button').click()
    for k in range(e1):#빨강
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color>.mdl-button').click()
    for k in range(e4):#파랑
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color+.input_color+.input_color>.mdl-button').click()
    for k in range(e5):#하늘
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color.input_color+.input_color+.input_color+.input_color>.mdl-button').click()
    for k in range(e3):#초록
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color+.input_color.input_color+.input_color+.input_color+.input_color>.mdl-button').click()
    for k in range(e6):#분홍
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color+.input_color>.mdl-button').click()
    for k in range(e7):#검정
            driver1.find_element_by_css_selector('.palette_stack>.input_color+.input_color+.input_color+.input_color+.input_color.input_color+.input_color+.input_color+.input_color>.mdl-button').click()
    time.sleep(7)
    color = driver1.find_element_by_css_selector('.mix_wrapper>.color_hex').text
    time.sleep(1)
    driver.quit() 
    time.sleep(1)
    song_color = color[1:]
    url3 = f'http://localhost:8082/testFlask/recieve_data?title={title}&singer={singer}&result={song_color}&emotion={pre}'
    return redirect(url3)
if __name__ == '__main__':
     app.run(host='127.0.1.1', port='5000')
        


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.1.1:5000/ (Press CTRL+C to quit)


In [43]:
app = Flask(__name__) 

@app.route('/test', methods=['GET','POST'])
def index():  
    global e1,e2,e3,e4,e5,e6,e7
    e1,e2,e3,e4,e5,e6,e7 = 0,0,0,0,0,0,0
    #title = request.args['title']
    #singer = request.args['singer']      
    if request.method == 'GET':
        title = request.args['title']
        singer = request.args['singer']        
    else:
        title = request.form['title']
        singer = request.form['singer']
    search=f'{singer}&{title}'
    url1 = f'https://www.genie.co.kr/search/searchMain?query={search}'      
    driver = wb.Chrome('chromedriver.exe',chrome_options=op)  
    driver.get(url1)
    time.sleep(1)
    driver.find_element_by_css_selector('.list>.link>.btn-basic').click()
    time.sleep(1)
    html = bs(driver.page_source, 'html.parser')
    a = html.select('#pLyrics p')[0]
    b = word_cv1.transform(a)
    pre = str (word_logi1.predict(b)[0])
    return pre
if __name__ == '__main__':
     app.run(host='127.0.1.1', port='5000')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.1.1:5000/ (Press CTRL+C to quit)
<ipython-input-43-b2885b6cd08a>:17: DeprecationWarning: use options instead of chrome_options
  driver = wb.Chrome('chromedriver.exe',chrome_options=op)
127.0.0.1 - - [19/Jun/2021 15:22:47] "GET /test?singer=아이유&title=라일락 HTTP/1.1" 200 -


In [38]:
url1 = f'https://www.genie.co.kr/search/searchMain?query=아이유&라일락'      
driver = wb.Chrome('chromedriver.exe',chrome_options=op)  
driver.get(url1)
time.sleep(1)
driver.find_element_by_css_selector('.list>.link>.btn-basic').click()
time.sleep(1)
html = bs(driver.page_source, 'html.parser')
a = html.select('#pLyrics p')
print(a)

<ipython-input-38-66532e937b79>:2: DeprecationWarning: use options instead of chrome_options
  driver = wb.Chrome('chromedriver.exe',chrome_options=op)


[<p>나리는 꽃가루에 눈이 따끔해 (아야)
눈물이 고여도 꾹 참을래
내 마음 한켠 비밀스런 오르골에 넣어두고서
영원히 되감을 순간이니까 

우리 둘의 마지막 페이지를 잘 부탁해
어느 작별이 이보다 완벽할까
Love me only till this spring

오 라일락 꽃이 지는 날 good bye
이런 결말이 어울려 
안녕 꽃잎 같은 안녕
하이얀 우리 봄날의 climax
아 얼마나 기쁜 일이야

Ooh ooh 
Love me only till this spring
봄바람처럼
Ooh ooh 
Love me only till this spring
봄바람처럼

기분이 달아 콧노래 부르네 (랄라)
입꼬리는 살짝 올린 채
어쩜 이렇게 하늘은 더 바람은 
또 완벽한 건지
오늘따라 내 모습 맘에 들어

처음 만난 그날처럼 예쁘다고 말해줄래
어느 이별이 이토록 달콤할까
Love resembles misty dream

오 라일락 꽃이 지는 날 good bye
이런 결말이 어울려
안녕 꽃잎 같은 안녕
하이얀 우리 봄날의 climax
아 얼마나 기쁜 일이야

Ooh ooh 
Love resembles misty dream
뜬구름처럼
Ooh ooh 
Love resembles misty dream
뜬구름처럼

너도 언젠가 날 잊게 될까
지금 표정과 오늘의 향기도
단잠 사이에 스쳐간
봄날의 꿈처럼

오 라일락 꽃이 지는 날 good bye
너의 대답이 날 울려
안녕 약속 같은 안녕
하이얀 우리 봄날에 climax
아 얼마나 기쁜 일이야

Ooh ooh 
Love me only untill this spring
봄바람처럼
Ooh ooh 
Love me only untill this spring
봄바람처럼

Ooh ooh 
Love resembles misty dream
뜬구름처럼
Ooh ooh 
Love resembles misty dream
뜬구름처럼</p>]


In [40]:
a[0]

<p>나리는 꽃가루에 눈이 따끔해 (아야)
눈물이 고여도 꾹 참을래
내 마음 한켠 비밀스런 오르골에 넣어두고서
영원히 되감을 순간이니까 

우리 둘의 마지막 페이지를 잘 부탁해
어느 작별이 이보다 완벽할까
Love me only till this spring

오 라일락 꽃이 지는 날 good bye
이런 결말이 어울려 
안녕 꽃잎 같은 안녕
하이얀 우리 봄날의 climax
아 얼마나 기쁜 일이야

Ooh ooh 
Love me only till this spring
봄바람처럼
Ooh ooh 
Love me only till this spring
봄바람처럼

기분이 달아 콧노래 부르네 (랄라)
입꼬리는 살짝 올린 채
어쩜 이렇게 하늘은 더 바람은 
또 완벽한 건지
오늘따라 내 모습 맘에 들어

처음 만난 그날처럼 예쁘다고 말해줄래
어느 이별이 이토록 달콤할까
Love resembles misty dream

오 라일락 꽃이 지는 날 good bye
이런 결말이 어울려
안녕 꽃잎 같은 안녕
하이얀 우리 봄날의 climax
아 얼마나 기쁜 일이야

Ooh ooh 
Love resembles misty dream
뜬구름처럼
Ooh ooh 
Love resembles misty dream
뜬구름처럼

너도 언젠가 날 잊게 될까
지금 표정과 오늘의 향기도
단잠 사이에 스쳐간
봄날의 꿈처럼

오 라일락 꽃이 지는 날 good bye
너의 대답이 날 울려
안녕 약속 같은 안녕
하이얀 우리 봄날에 climax
아 얼마나 기쁜 일이야

Ooh ooh 
Love me only untill this spring
봄바람처럼
Ooh ooh 
Love me only untill this spring
봄바람처럼

Ooh ooh 
Love resembles misty dream
뜬구름처럼
Ooh ooh 
Love resembles misty dream
뜬구름처럼</p>